## Parameters

In [1]:
vocab_size           = 5000
hide_most_frequently = 0

epochs               = 10
batch_size           = 512
fit_verbosity        = 1

In [2]:
run_dir = "run/"

In [3]:
import os
os.environ['KERAS_BACKEND'] = "torch"
import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

2024-03-23 00:10:37.274326: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 00:10:37.328454: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 00:10:37.328511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 00:10:37.329469: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 00:10:37.342552: I tensorflow/core/platform/cpu_feature_guar

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from modules import encoding

## Retrieve data


In [6]:
df_match = pd.read_csv("../data/SAbDab/data.csv", sep=";")
# df_match[["ab_id", "ab_type"]] = df_match["ab"].str.split('|', n=1, expand=True)
# df_match[["ag_id", "ag_type"]] = df_match["ag"].str.split('|', n=1, expand=True)
df_match.head()

,ab,ag,interaction
0,5kel|ab,5kel|ag,1
1,5kel|ab,6cwt|ag,0
2,5kel|ab,4fp8|ag,0
3,5kel|ab,4yjz|ag,0
4,5kel|ab,6j15|ag,0


In [7]:
df_seq = pd.read_csv("../data/SAbDab/sequences.csv", sep=";")
df_seq.head()

,seq_id,specie,sequence
0,5kel|ag,Zaire ebolavirus (strain Mayinga-76) (128952),IPLGVIHNSTLQVSDVDKLVCRDKLSSTNQLRSVGLNLEGNGVATD...
1,5kel|ag,Zaire ebolavirus (128952),EAIVNAQPKCNPNLHYWTTQDEGAAIGLAWIPYFGPAAEGIYTEGL...
2,5kel|ab,Homo sapiens (9606),EVQLQESGGGLMQPGGSMKLSCVASGFTFSNYWMNWVRQSPEKGLE...
3,5kel|ab,Homo sapiens (9606),DIQMTQSPASLSVSVGETVSITCRASENIYSSLAWYQQKQGKSPQL...
4,5kel|ab,Homo sapiens (9606),DVKLLESGGGLVQPGGSLKLSCAASGFSLSTSGVGVGWFRQPSGKG...


In [8]:
seq = df_seq["sequence"]
encoder = encoding.alphabet_one_hot(alphabet=encoding.AMINO_ACID_ALPHABET)

In [10]:
encoded = encoding.one_hot_encode_sequence("MYA", encoder, 100)

In [11]:
vector_size = 1500
seq_encoded = encoding.one_hot_encoder(seq, encoder, )

In [12]:
ALPHABET_SIZE = len(encoding.AMINO_ACID_ALPHABET)

### Split dataset

In [13]:
df_seq[["seq_rcpb", "seq_type"]] = df_seq["seq_id"].str.split('|',  n=1, expand=True)
ordinal_encoder = OrdinalEncoder()
enc_seq_type = ordinal_encoder.fit_transform(df_seq[["seq_type"]])

In [14]:
X = seq_encoded
y = enc_seq_type

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model

In [16]:
model = keras.Sequential(name="abag_classifier")

model.add(keras.layers.Input(shape=(vector_size, ALPHABET_SIZE)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense( 32, activation='relu'))
model.add(keras.layers.Dense( 32, activation='relu'))
model.add(keras.layers.Dense( 1,  activation='sigmoid'))

    
model.compile(optimizer = 'rmsprop',
                  loss      = 'binary_crossentropy',
                  metrics   = ['accuracy'])

model.summary()

Model: "abag_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 33000)             0         
                                                                 
 dense (Dense)               (None, 32)                1056032   
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1057121 (4.03 MB)
Trainable params: 1057121 (4.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-03-23 00:10:42.326569: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## Train the model

### Add callback

In [17]:
os.makedirs(f'{run_dir}/models', mode=0o750, exist_ok=True)
save_dir = f'{run_dir}/models/best_model.keras'

savemodel_callback = keras.callbacks.ModelCheckpoint(filepath=save_dir, monitor='val_accuracy', mode='max', save_best_only=True)

### Train it

In [18]:
%%time

history = model.fit(X_train,
                    y_train,
                    epochs          = epochs,
                    batch_size      = batch_size,
                    validation_data = (X_test, y_test),
                    verbose         = fit_verbosity,
                    callbacks       = [savemodel_callback])

Epoch 1/10
5/5 [==============================] - 1s 138ms/step - loss: 0.5564 - accuracy: 0.7572 - val_loss: 0.4492 - val_accuracy: 0.8451
Epoch 2/10
5/5 [==============================] - 0s 63ms/step - loss: 0.3829 - accuracy: 0.8702 - val_loss: 0.3944 - val_accuracy: 0.8600
Epoch 3/10
5/5 [==============================] - 0s 62ms/step - loss: 0.3175 - accuracy: 0.8862 - val_loss: 0.4021 - val_accuracy: 0.8682
Epoch 4/10
5/5 [==============================] - 0s 50ms/step - loss: 0.2805 - accuracy: 0.9056 - val_loss: 0.3625 - val_accuracy: 0.8550
Epoch 5/10
5/5 [==============================] - 0s 52ms/step - loss: 0.2609 - accuracy: 0.9101 - val_loss: 0.3545 - val_accuracy: 0.8682
Epoch 6/10
5/5 [==============================] - 0s 61ms/step - loss: 0.2269 - accuracy: 0.9229 - val_loss: 0.3641 - val_accuracy: 0.8699
Epoch 7/10
5/5 [==============================] - 0s 50ms/step - loss: 0.2214 - accuracy: 0.9213 - val_loss: 0.3525 - val_accuracy: 0.8649
Epoch 8/10
5/5 [==========

In [21]:
X_train.shape

(2426, 1500, 22)

In [20]:
y_train

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]])